In [82]:
from asl.ARMParser import get_parser
from asl.ARMAST import Instruction
from ARMIntrinsics import EncodingFields
# from ARMTypes import *
from ARMAST import *
import json

with open("intr.json", 'r') as fi:
    I = json.load(fi)
with open("asl/arm_instrs.sexpr") as f:
    data = f.read()
parser = get_parser()
Instrs: List[Instruction] = parser.parse(data)


In [9]:
enc = "aarch64_vector_arithmetic_binary_uniform_add_wrapping_single_simd"
field = EncodingFields[enc]
op = None
encoding = None
for i in Instrs:
    for j in i.instEncodings:
        if j.encName == enc:
            op = i.instExecute
            encoding = j
            print(op, encoding)
            break


[StmtCall(qid=QualifiedIdentifier(archQualifier='ANY', id='CheckFPAdvSIMDEnabled64'), exprs=[]), StmtVarDeclInit(symboldecl=SymbolDecl(id='operand1', ty=TypeFun(id='bits', expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='datasize')))), expr=ExprIndex(expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='V')), slices=[SliceSingle(expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='n')))])), StmtVarDeclInit(symboldecl=SymbolDecl(id='operand2', ty=TypeFun(id='bits', expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='datasize')))), expr=ExprIndex(expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='V')), slices=[SliceSingle(expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='m')))])), StmtVarsDecl(ids=['result'], ty=TypeFun(id='bits', expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='datasize')))), StmtVarsDecl(ids=['element1'], ty=TypeFun(id='bits', expr=ExprVarRef(qid=QualifiedIdentifier(archQua

In [148]:


opcd = genPossibleOpcode({'Q': 'x', 'U': '1', 'size': 'xx', 'Rm': 'xxxxx',
                          'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'})
print(len(opcd))
for i in opcd:
    D = DecodeContext(i)
    D.walk(newAST)
    if D.undefined:
        print(i, 'undefined')
    else:
        print(i, D.result)


8
{'Q': '0', 'U': '1', 'size': '0', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'} {'d': 0, 'n': 0, 'm': 0, 'esize': 8, 'datasize': 64, 'elements': 8, 'sub_op': True}
{'Q': '1', 'U': '1', 'size': '0', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'} {'d': 0, 'n': 0, 'm': 0, 'esize': 8, 'datasize': 128, 'elements': 16, 'sub_op': True}
{'Q': '0', 'U': '1', 'size': '1', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'} {'d': 0, 'n': 0, 'm': 0, 'esize': 16, 'datasize': 64, 'elements': 4, 'sub_op': True}
{'Q': '1', 'U': '1', 'size': '1', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'} {'d': 0, 'n': 0, 'm': 0, 'esize': 16, 'datasize': 128, 'elements': 8, 'sub_op': True}
{'Q': '0', 'U': '1', 'size': '10', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'} {'d': 0, 'n': 0, 'm': 0, 'esize': 32, 'datasize': 64, 'elements': 2, 'sub_op': True}
{'Q': '1', 'U': '1', 'size': '10', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx

In [192]:
class CaseBase(ASTNode):
    body: List[ASTNode]
    id: str

    def __init__(self, body: List[ASTNode], id: str):
        self.body, self.id = body, id

    def __repr__(self):
        return f"CaseBase({self.body}, {self.id})"


class Case(CaseBase):
    val: ASTNode
    body: List[ASTNode]
    id: str

    def __init__(self, val: ASTNode, body: List[ASTNode], id: str):
        self.val, self.body, self.id = val, body, id

    def __repr__(self):
        return f"Case({self.val}, {self.body}, {self.id})"


class Match(ASTNode):
    val: ASTNode
    cases: List[CaseBase]
    id: str

    def __init__(self, val: ASTNode, cases: List[CaseBase], id: str):
        self.val, self.cases, self.id = val, cases, id

    def __repr__(self):
        return f"Match({self.val}, {self.cases}, {self.id})"


def ASTShrink(AST):
    if isinstance(AST, list):
        w = [ASTShrink(i) for i in AST]
        return [i for i in w if i is not None]
    if isinstance(AST, asl.StmtCall):
        return None  # Assume StmtCall useless
    elif isinstance(AST, asl.StmtVarDeclInit):
        return VarDeclInit(ASTShrink(AST.symboldecl), ASTShrink(AST.expr), GenUniqueID())
    elif isinstance(AST, asl.SymbolDecl):
        return VarsDecl([Var(AST.id, GenUniqueID())], ASTShrink(AST.ty), GenUniqueID())
    elif isinstance(AST, asl.StmtVarsDecl):
        return VarsDecl([Var(i, GenUniqueID()) for i in AST.ids], ASTShrink(AST.ty), GenUniqueID())
    elif isinstance(AST, asl.TypeFun):  # Only Returns Type width
        if AST.id == "bits":
            return ASTShrink(AST.expr)
        assert False
    elif isinstance(AST, asl.ExprVarRef):
        return Var(ASTShrink(AST.qid), GenUniqueID())
    elif isinstance(AST, asl.LValVarRef):
        return Var(ASTShrink(AST.qid), GenUniqueID())
    elif isinstance(AST, asl.LValArrayIndex):
        return ArrayIndex(ASTShrink(AST.lvalexpr), ASTShrink(AST.slices), GenUniqueID())
    elif isinstance(AST, asl.QualifiedIdentifier):
        return AST.id
    elif isinstance(AST, asl.ExprIndex):
        return ArrayIndex(ASTShrink(AST.expr), ASTShrink(AST.slices), GenUniqueID())
    elif isinstance(AST, asl.ExprSlice):
        assert len(AST.slices) == 1
        return ArraySlice(ASTShrink(AST.expr), ASTShrink(AST.slices[0]), GenUniqueID())
    elif isinstance(AST, asl.SliceRange):
        return [ASTShrink(AST.expr0), ASTShrink(AST.expr1)]
    elif isinstance(AST, asl.SliceSingle):
        return [ASTShrink(AST.expr)]
    elif isinstance(AST, asl.StmtFor):
        return For(Var(AST.id, GenUniqueID()), ASTShrink(AST.expr0), ASTShrink(AST.expr1), ASTShrink(AST.stmts), 1, GenUniqueID())
    elif isinstance(AST, asl.ExprLitInt):
        return Number(AST.integer)
    elif isinstance(AST, asl.ExprBinOp):
        return BinaryExpr(AST.binop[1:-1], ASTShrink(AST.lhs), ASTShrink(AST.rhs), GenUniqueID())
    elif isinstance(AST, asl.ExprUnOp):
        return UnaryExpr(AST.unop[1:-1], ASTShrink(AST.expr), GenUniqueID())
    elif isinstance(AST, asl.StmtAssign):
        return Update(ASTShrink(AST.lvalexpr), ASTShrink(AST.expr))
    elif isinstance(AST, asl.ExprCall):
        return Call(ASTShrink(AST.qid), ASTShrink(AST.exprs), GenUniqueID())
    elif isinstance(AST, asl.StmtIf):
        if AST.maybe_stmts != asl.Nothing():
            if len(AST.stmtifcases) == 1:
                return IfElse(ASTShrink(AST.stmtifcases[0].expr), ASTShrink(AST.stmtifcases[0].stmts), ASTShrink(AST.maybe_stmts), GenUniqueID())
            else:
                print(AST)
                assert False
        else:
            if len(AST.stmtifcases) == 1:
                return If(ASTShrink(AST.stmtifcases[0].expr), ASTShrink(AST.stmtifcases[0].stmts), GenUniqueID())
            else:
                print(AST)
                assert False
    elif isinstance(AST, asl.TypeRef):
        return 32
    elif isinstance(AST, asl.ExprLitBin):
        return BitVec(AST.bitvector)
    elif isinstance(AST, asl.StmtUndefined):
        return Undefiend()
    elif isinstance(AST, asl.ExprIf):
        return IfElse(ASTShrink(AST.exprtest), ASTShrink(AST.exprresult), ASTShrink(AST.exprelse), GenUniqueID())
    elif isinstance(AST, asl.ExprUnknown):
        return Number(0)
    elif isinstance(AST, asl.StmtCase):
        return Match(ASTShrink(AST.expr), ASTShrink(AST.casealternatives), GenUniqueID())
    elif isinstance(AST, asl.CaseWhen):
        return Case(ASTShrink(AST.casepatterns), ASTShrink(AST.stmts), GenUniqueID())
    elif isinstance(AST, asl.CaseOtherwise):
        return CaseBase(ASTShrink(AST.stmts), GenUniqueID())
    elif isinstance(AST, asl.CasePatternMask):
        return BitVec(AST.mask)
    elif isinstance(AST, asl.CasePatternBin):
        return BitVec(AST.bitvector)
    elif isinstance(AST, asl.CasePatternInt):
        return Number(AST.integer)
    elif isinstance(AST, asl.ExprLitMask):
        return BitVec(AST.mask)
    else:
        print(AST)
        assert False


zzzz = set(EncodingFields.keys())
map2AST = {}
for i in Instrs:
    for j in i.instEncodings:
        if j.encName in zzzz:
            print(j.encName)
            # print(j.encDecode)
            wholeCode = j.encDecode
            if type(j.encDecode) != asl.Nothing:
                wholeCode += i.instPostDecode
            map2AST[j.encName] = ((i.instExecute), ASTShrink(wholeCode))
assert len(map2AST) == len(zzzz)


aarch64_vector_arithmetic_binary_uniform_add_fp16
aarch64_vector_arithmetic_binary_uniform_add_fp
aarch64_vector_arithmetic_binary_disparate_mul_dmacc_sisd
aarch64_vector_arithmetic_binary_disparate_mul_dmacc_simd
aarch64_vector_arithmetic_unary_special_recip_int
aarch64_memory_vector_multiple_no_wb
aarch64_memory_vector_multiple_post_inc
aarch64_vector_shift_right_narrow_logical
aarch64_vector_arithmetic_binary_disparate_add_sub_wide
aarch64_vector_arithmetic_binary_uniform_max_min_single
aarch64_vector_arithmetic_unary_fp16_round
aarch64_vector_arithmetic_unary_float_round
aarch64_vector_arithmetic_binary_disparate_mul_accum
aarch64_vector_arithmetic_binary_uniform_sub_fp16_sisd
aarch64_vector_arithmetic_binary_uniform_sub_fp_sisd
aarch64_vector_arithmetic_binary_uniform_sub_fp16_simd
aarch64_vector_arithmetic_binary_uniform_sub_fp_simd
aarch64_vector_crypto_sm3_sm3tt1a
aarch64_vector_arithmetic_unary_fp16_conv_float_bulk_sisd
aarch64_vector_arithmetic_unary_float_conv_float_bulk_sis

In [175]:
def genPossibleOpcode(fields):
    qwq = [fields]
    for i, v in fields.items():
        if i.startswith('R'):
            continue
        if 'x' in v:
            # print(fields[i])
            # assert fields[i] == 'x'*len(fields[i])
            ln = fields[i].count('x')
            nqwq = []
            zz = fields[i]
            for z in range(2**ln):
                z = bin(z)[2:]
                z = '0'*(ln-len(z)) + z
                zz = fields[i]
                for j in z:
                    zz = zz.replace('x', j, 1)
                nqwq = nqwq + [dict(q, **{i: zz})for q in qwq]
            qwq = nqwq
    return qwq


opcd = genPossibleOpcode({'Q': 'x', 'U': '1', 'size': 'xx', 'Rm': 'xxxxx',
                          'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'})
for i in opcd:
    print(i)


{'Q': '0', 'U': '1', 'size': '00', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
{'Q': '1', 'U': '1', 'size': '00', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
{'Q': '0', 'U': '1', 'size': '01', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
{'Q': '1', 'U': '1', 'size': '01', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
{'Q': '0', 'U': '1', 'size': '10', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
{'Q': '1', 'U': '1', 'size': '10', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
{'Q': '0', 'U': '1', 'size': '11', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
{'Q': '1', 'U': '1', 'size': '11', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}


In [198]:

def LowestSetBit(x: str):
    N = len(x)
    for i in range(N):
        if x[N-i-1] == '1':
            return i
    return N


class DecodeContext:
    undefined = None
    symbolnotfound = None

    def __init__(self, fields):
        self.fields = fields
        self.result = {}

    def EQ(self, a, b):
        return a == b

    def symtable(self):
        # print(self.fields, self.result)
        return dict(self.fields, **self.result)

    def lookup(self, name):
        Z = self.symtable()
        if name not in Z:
            raise LookupError(f"Symbol {name} not found")
        return self.symtable()[name]

    def walkConstExprLV(self, AST: ASTNode):
        if isinstance(AST, Var):
            if AST.name.startswith("R"):
                assert False
            else:
                return AST.name
        else:
            print(AST)
            assert False

    def walkConstExprRV(self, AST: ASTNode):
        if isinstance(AST, Var):
            if AST.name.startswith("R"):
                return '00000'  # We just don't care about register
            else:
                return self.lookup(AST.name)
        elif isinstance(AST, Call):
            if AST.funcname == "UInt":
                return int(self.walkConstExprRV(AST.args[0]), 2)
            elif AST.funcname.startswith("Have"):
                return True
            elif AST.funcname == "LowestSetBit":
                return LowestSetBit(self.walkConstExprRV(AST.args[0]))
            print(AST)
            assert False
        elif isinstance(AST, BitVec):
            return AST.bv
        elif isinstance(AST, ArraySlice):
            obj = self.walkConstExprRV(AST.bv)
            assert type(obj) == str
            sliceRV = [self.walkConstExprRV(i) for i in AST.slices]
            wid = len(obj)
            if len(sliceRV) == 1:
                return obj[wid-sliceRV[0]-1]
            if len(sliceRV) == 2:
                return obj[wid-sliceRV[0]-1:wid-sliceRV[1]]
            print(sliceRV)
            assert False
        elif isinstance(AST, Number):
            return AST.val
        elif isinstance(AST, IfElse):
            cond = self.walkConstExprRV(AST.cond)
            assert type(cond) == bool
            if cond:
                return self.walkConstExprRV(AST.then)
            else:
                return self.walkConstExprRV(AST.otherwise)
        elif isinstance(AST, UnaryExpr):
            a = self.walkConstExprRV(AST.a)
            if AST.op == "!":
                assert type(a) == bool
                return not a
            else:
                print(AST)
                assert False
        elif isinstance(AST, BinaryExpr):
            a = self.walkConstExprRV(AST.a)
            b = self.walkConstExprRV(AST.b)
            if AST.op == "<<":
                assert type(a) == int and type(b) == int
                return a << b
            elif AST.op == ">>":
                assert type(a) == int and type(b) == int
                return a >> b
            elif AST.op == "&&":
                assert type(a) == bool and type(b) == bool
                return a and b
            elif AST.op == "||":
                assert type(a) == bool and type(b) == bool
                return a or b
            # elif AST.op == "^":
            #     return a ^ b
            elif AST.op == "+":
                assert type(a) == int and type(b) == int
                return a + b
            elif AST.op == "-":
                assert type(a) == int and type(b) == int
                return a - b
            elif AST.op == "*":
                assert type(a) == int and type(b) == int
                return a*b
            elif AST.op == "DIV":
                assert type(a) == int and type(b) == int
                assert a % b == 0
                return a//b
            elif AST.op == ":":
                assert type(a) == str and type(b) == str
                return a+b
            elif AST.op == ">":
                assert type(a) == int and type(b) == int
                return a > b
            elif AST.op == "==":
                assert type(a) == type(b)
                if type(a) == str:
                    assert 'x' not in a and 'x' not in b
                return self.EQ(a, b)
            elif AST.op == "!=":
                assert type(a) == type(b)
                if type(a) == str:
                    assert 'x' not in a and 'x' not in b
                return not self.EQ(a, b)
            else:
                print(AST)
                assert False
        else:
            print(AST)
            assert False

    def walkConstExprStmt(self, AST: ASTNode):
        if isinstance(AST, list):
            for i in AST:
                self.walkConstExprStmt(i)
        elif isinstance(AST, VarDeclInit):
            self.result[self.walkConstExprLV(
                AST.decl.ids[0])] = self.walkConstExprRV(AST.expr)
        elif isinstance(AST, Update):
            self.result[self.walkConstExprLV(
                AST.lhs)] = self.walkConstExprRV(AST.rhs)
        elif isinstance(AST, If):
            if self.walkConstExprRV(AST.cond):
                self.walkConstExprStmt(AST.then)
        elif isinstance(AST, Undefiend):
            raise NotImplementedError("Such field reaches undefined")
        elif isinstance(AST, VarsDecl):
            pass
        elif isinstance(AST, Match):
            for c in AST.cases:
                if isinstance(c, Case):  # not default
                    # print(c)
                    if len(c.val) == 1:
                        if self.EQ(AST.val, self.walkConstExprRV(c.val[0])):
                            self.walkConstExprStmt(c.body)
                            break
                    else:
                        assert False
                else:
                    self.walkConstExprStmt(c.body)
                    break
        else:
            print(AST)
            assert False

    def walk(self, AST: ASTNode):
        try:
            return self.walkConstExprStmt(AST)
        except LookupError as e:
            self.symbolnotfound = e
            return {}
        except NotImplementedError as e:
            self.undefined = e
            return {}


for encodings, f in EncodingFields.items():
    # print(encodings)
    if encodings == "aarch64_vector_logical":
        continue
    print(encodings, f[1])
    opcd = genPossibleOpcode(f[1])
    for i in opcd:
        D = DecodeContext(i)
        # print(encodings, i)
        D.walk(map2AST[encodings][1])
        if D.undefined:
            pass
            # print(i, 'undefined')
        elif D.symbolnotfound:
            print(encodings, D.symbolnotfound)
            # print(i, 'symbolnotfound')
        else:
            pass
            # print(i, D.result)


aarch64_vector_arithmetic_unary_diff_neg_int_sisd {'U': '1', 'size': 'xx', 'opcode': '01011', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
aarch64_vector_arithmetic_unary_diff_neg_int_simd {'Q': 'x', 'U': '1', 'size': 'xx', 'opcode': '01011', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
aarch64_vector_arithmetic_binary_uniform_add_wrapping_single_sisd {'U': '1', 'size': 'xx', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
aarch64_vector_arithmetic_binary_uniform_add_wrapping_single_simd {'Q': 'x', 'U': '1', 'size': 'xx', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
aarch64_vector_arithmetic_binary_disparate_add_sub_narrow {'Q': 'x', 'U': '0', 'size': 'xx', 'Rm': 'xxxxx', 'o1': '1', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
aarch64_vector_reduce_add_sisd {'U': '0', 'size': 'xx', 'opcode': '11011', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
aarch64_vector_arithmetic_binary_uniform_add_wrapping_pair {'Q': 'x', 'U': '0', 'size': 'xx', 'Rm': 'xxxxx', 'opcode': '10111', 'Rn': 'xxxxx', 'Rd': 'xxxxx'}
aarch64_ve

AssertionError: 